In [473]:
import requests
import json
import pandas as pd
import time
from tqdm import tqdm_notebook

In [494]:
naver_data = pd.read_csv('./data/Cheongcheon_up_to_date_detail.csv')
landbook = pd.read_csv('./data/landbook.csv')

In [495]:
naver_location = naver_data[['longitude','latitude','pnu']].drop_duplicates()
# naver_location['naver_pnu'] = naver_location['pnu']
naver_location = naver_location.reset_index(drop = True)
naver_location.columns = ['naver_longitude', 'naver_latitude', 'naver_pnu']

In [496]:
del landbook['naver_pnu']

In [497]:
df = pd.merge(landbook,naver_location, on= ['naver_longitude','naver_latitude'], how = 'outer')

In [498]:
search_df = df[df['id'].isna()==True]

In [479]:
result_landbook = []
for idx, key in enumerate(tqdm_notebook(search_df.index)):
    url = 'https://api2.landbook.net/point_polygons?lon={lon}&lat={lat}'.format(lon = search_df['naver_longitude'][key], lat = search_df['naver_latitude'][key])
    r = requests.get(url, timeout = 5)
    result_landbook. append([json.loads(r.text)['id'], search_df['naver_longitude'][key], search_df['naver_latitude'][key],search_df['naver_pnu'][key]])
    time.sleep(2)

C:\Users\gongdan\AppData\Local\Temp/ipykernel_9708/826260398.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx, key in enumerate(tqdm_notebook(search_df.index)):


  0%|          | 0/1 [00:00<?, ?it/s]

In [480]:
jibeon = pd.DataFrame(result_landbook)
jibeon.columns = ['pnu','longitude','latitude','naver_pnu']
jibeon = jibeon.astype({'pnu':'int64','naver_pnu':'int64'})

In [ ]:
## 기존 landbook 데이터와 신규로 찾을 데이터 비교 과정 필요
# need_search_jibeon = 

In [481]:
result_jibeon = pd.DataFrame()
for idx, item in tqdm.tqdm_notebook(enumerate(jibeon['pnu'])):
    
    url = 'https://api2.landbook.net/parcels/{num}'.format(num = item)
    r = requests.get(url, timeout = 5)
    try:
        info = json.loads(r.text)['parcel_info']
        info.pop('geometry')
        info.pop('building_list')
        info.pop('other_parcel')
        info = dict(info)
        temp = pd.DataFrame(info,  index = [0])
        result_jibeon = pd.concat([result_jibeon, temp])
        time.sleep(2)
    except:
        print(idx, "error", item)
result_jibeon = result_jibeon.reset_index(drop = True)

C:\Users\gongdan\AppData\Local\Temp/ipykernel_9708/2791178482.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx, item in tqdm.tqdm_notebook(enumerate(jibeon['pnu'])):


0it [00:00, ?it/s]

In [482]:
result_jibeon = result_jibeon.reset_index(drop = True)

In [483]:
result_jibeon[['naver_longitude','naver_latitude','naver_pnu']] = jibeon[['longitude','latitude','naver_pnu']]

In [484]:
landbook = pd.read_csv('./data/landbook.csv')

In [485]:
result = pd.concat([landbook, result_jibeon])

In [486]:
result.to_csv('./data/landbook.csv', index = False, encoding = "utf-8-sig")